In [2]:
# ✅ 1. Setup
!rm -rf agrix_beta_2025
!git clone https://github.com/smwaingeni-ai/agrix_beta_2025.git
!ls agrix_beta_2025/assets/crops

import pandas as pd
from datetime import datetime

# ✅ Step 2: Load CSV data
csv_path = "agrix_beta_2025/assets/data/soil_map_africa.csv"

try:
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip().str.lower()

    required_cols = ['region', 'soil_type', 'severity', 'treatment', 'suitable_crops', 'image_name']
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"❌ Missing column: {col}")

    print("✅ CSV loaded with columns:", list(df.columns))
except Exception as e:
    print(f"❌ Failed to load CSV: {e}")
    raise

Cloning into 'agrix_beta_2025'...
remote: Enumerating objects: 2014, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 2014 (delta 126), reused 37 (delta 37), pack-reused 1654 (from 1)
Receiving objects: 100% (2014/2014), 77.47 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (933/933), done.
Healthy  Maize___Leaf_Spot  Tomato___Bacterial_spot
✅ CSV loaded with columns: ['region', 'soil_type', 'severity', 'treatment', 'suitable_crops', 'image_name']


In [3]:
# ✅ Step 3: Simple rule-based filter engine

def query_soil_advice(region_input: str, soil_type_input: str, df_source: pd.DataFrame):
    region_input = region_input.lower().strip()
    soil_input = soil_type_input.lower().strip()

    matches = df_source[
        df_source['region'].str.lower().str.contains(region_input) &
        df_source['soil_type'].str.lower().str.contains(soil_input)
    ]

    return matches


In [4]:
# ✅ Step 4: Sample user inputs
user_queries = [
    {"region": "Central", "soil_type": "Sandy"},
    {"region": "Luapula", "soil_type": "Peaty"},
    {"region": "Southern", "soil_type": "Acidic"},
    {"region": "Western", "soil_type": "Waterlogged"},
    {"region": "Unknown", "soil_type": "Unknown"}
]

results_log = []

for query in user_queries:
    print(f"\n🔍 Input: Region = {query['region']}, Soil Type = {query['soil_type']}")
    matches = query_soil_advice(query["region"], query["soil_type"], df)

    if not matches.empty:
        for _, row in matches.iterrows():
            print(f"✅ Match Found:")
            print(f"🌍 Region: {row['region']}")
            print(f"🪴 Soil Type: {row['soil_type']}")
            print(f"⚠️ Severity: {row['severity']}")
            print(f"💡 Treatment: {row['treatment']}")
            print(f"🌾 Suitable Crops: {row['suitable_crops']}")
            print(f"🖼️ Image: {row['image_name']}")
            print()

            results_log.append({
                "timestamp": datetime.now().isoformat(),
                "region": row['region'],
                "soil_type": row['soil_type'],
                "severity": row['severity'],
                "treatment": row['treatment'],
                "suitable_crops": row['suitable_crops'],
                "image": row['image_name']
            })
    else:
        print("❌ No matching recommendation found.")



🔍 Input: Region = Central, Soil Type = Sandy
✅ Match Found:
🌍 Region: Central
🪴 Soil Type: Sandy
⚠️ Severity: Low
💡 Treatment: Apply cow dung + lime
🌾 Suitable Crops: Cassava; Millet; Sorghum
🖼️ Image: sandy.jpg


🔍 Input: Region = Luapula, Soil Type = Peaty
✅ Match Found:
🌍 Region: Luapula
🪴 Soil Type: Peaty
⚠️ Severity: Low
💡 Treatment: Drain and apply balanced fertilizer
🌾 Suitable Crops: Rice; Taro
🖼️ Image: peaty.jpg


🔍 Input: Region = Southern, Soil Type = Acidic
✅ Match Found:
🌍 Region: Southern
🪴 Soil Type: Acidic
⚠️ Severity: Low
💡 Treatment: Apply lime and rotate with legumes
🌾 Suitable Crops: Sweet Potatoes; Beans
🖼️ Image: acidic.jpg


🔍 Input: Region = Western, Soil Type = Waterlogged
✅ Match Found:
🌍 Region: Western
🪴 Soil Type: Waterlogged
⚠️ Severity: Very Low
💡 Treatment: Raise planting beds and improve drainage
🌾 Suitable Crops: Rice; Sugarcane
🖼️ Image: waterlogged.jpg


🔍 Input: Region = Unknown, Soil Type = Unknown
❌ No matching recommendation found.


In [5]:
# ✅ Step 5: Save diagnosis log (optional)
if results_log:
    df_log = pd.DataFrame(results_log)
    log_file = "soil_diagnosis_log.csv"
    df_log.to_csv(f"agrix_beta_2025/assets/data/{log_file}", index=False)
    print(f"📝 Diagnosis log saved to: {log_file}")
else:
    print("📭 No results to log.")


📝 Diagnosis log saved to: soil_diagnosis_log.csv
